# **Custom Samsung Chatbot Recommender System**

### Links to the repository and code:

- [Repository](https://github.com/IISF-SIF/SamsungPrismHack)
- [Code](https://github.com/IISF-SIF/SamsungPrismHack/blob/main/samsungrecommender.ipynb)


 # Setting Up Environment and Importing Libraries

### *Here we setup the intent classifier from the custom created samsung prompt dataset that is now open-source on Kaggle. The dataset was created across 10 different classes with a total of 967 diverse datapoints. The dataset was created using GPT 3.5, the goal of this recommender system is increase ease-of-use among users when using Samsung IoT Devices*

### **Intent Dataset Creation using BERTClassifer From User History**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Importing necessary libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# Importing the os module to interact with the operating system
import os

# Iterating through files in the input directory and printing their names
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/samsungrecommend/RecommenderSamsungDevice - Sheet1.csv


In [2]:
# Installing necessary libraries using pip
!pip install tqdm transformers torch

# Importing required libraries
import pandas as pd  # For data manipulation
import torch  # PyTorch library for deep learning
from tqdm.notebook import tqdm  # For progress bar

# Importing BERT tokenizer from Hugging Face Transformers library
from transformers import BertTokenizer

# Importing TensorDataset for creating datasets
from torch.utils.data import TensorDataset

# Importing BERT model for sequence classification from Transformers library
from transformers import BertForSequenceClassification

# Data Loading and Preprocessing

In [3]:
# Reading the CSV file into a pandas DataFrame
data = pd.read_csv('/kaggle/input/samsungrecommend/RecommenderSamsungDevice - Sheet1.csv')

# Displaying the first few rows of the DataFrame
data.head()

# Counting the occurrences of each unique value in the 'Query' column
data['Query'].value_counts()

# Extracting unique values from the 'Query' column and assigning them to the variable 'possible_labels'
possible_labels = data.Query.unique()

In [4]:
# Creating an empty dictionary to store label mappings
label_dict = {}

# Iterating through the unique labels in possible_labels along with their index using enumerate
for index, possible_label in enumerate(possible_labels):
    # Assigning each unique label to its corresponding index in the dictionary
    label_dict[possible_label] = index

# Displaying the label dictionary
label_dict

{'Phone': 0,
 'Tab': 1,
 'TV': 2,
 'AC': 3,
 'Wash': 4,
 'Fridge': 5,
 'Vacuum': 6,
 'Dish': 7,
 'Micro': 8,
 'Watch': 9}

# Data Preprocessing and Splitting

In [5]:
# Replacing labels in the 'Query' column with their corresponding indices from the label_dict dictionary
data['label'] = data.Query.replace(label_dict)

# Importing train_test_split function from scikit-learn
from sklearn.model_selection import train_test_split

# Splitting the data into training and validation sets
# X_train, X_val: Indices of training and validation data
# y_train, y_val: Labels corresponding to the training and validation data
# test_size: Proportion of the dataset to include in the validation split
# random_state: Seed for random number generation for reproducibility
# stratify: Ensures that the distribution of labels is similar in both training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

# Adding a new column 'data_type' to the DataFrame to indicate whether each row belongs to the training or validation set
data['data_type'] = ['not_set']*data.shape[0]

# Marking rows corresponding to training and validation indices with 'train' and 'val' respectively
data.loc[X_train, 'data_type'] = 'train'
data.loc[X_val, 'data_type'] = 'val'

# Grouping the data by 'Query', 'label', and 'data_type' columns and counting the occurrences of each group
data.groupby(['Query', 'label', 'data_type']).count()

/tmp/ipykernel_72/1309740842.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['label'] = data.Query.replace(label_dict)


Input
Query  label data_type       
AC     3     train         85
             val           15
Dish   7     train         85
             val           15
Fridge 5     train         85
             val           15
Micro  8     train         85
             val           15
Phone  0     train         85
             val           15
TV     2     train         85
             val           15
Tab    1     train         85
             val           15
Vacuum 6     train         85
             val           15
Wash   4     train         85
             val           15
Watch  9     train         85
             val           15

# Tokenization and Encoding

In [6]:
# Importing the BERT tokenizer from the Hugging Face Transformers library and loading the 'bert-base-uncased' model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
# Tokenizing and encoding the training data
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].Input.values,  # Extracting input text data for the training set
    add_special_tokens=True,  # Adding special tokens like [CLS] and [SEP]
    return_attention_mask=True,  # Returning attention masks to indicate which tokens are padding tokens
    pad_to_max_length=True,  # Padding sequences to the maximum length
    max_length=256,  # Maximum sequence length
    return_tensors='pt'  # Returning PyTorch tensors
)

# Tokenizing and encoding the validation data
encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='val'].Input.values,  # Extracting input text data for the validation set
    add_special_tokens=True,  # Adding special tokens like [CLS] and [SEP]
    return_attention_mask=True,  # Returning attention masks to indicate which tokens are padding tokens
    pad_to_max_length=True,  # Padding sequences to the maximum length
    max_length=256,  # Maximum sequence length
    return_tensors='pt'  # Returning PyTorch tensors
)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2645: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


# Dataset Creation

In [7]:
# Extracting input IDs, attention masks, and labels for the training set
input_ids_train = encoded_data_train['input_ids']  # Extracting input IDs
attention_masks_train = encoded_data_train['attention_mask']  # Extracting attention masks
labels_train = torch.tensor(data[data.data_type=='train'].label.values)  # Extracting labels and converting to PyTorch tensor

# Extracting input IDs, attention masks, and labels for the validation set
input_ids_val = encoded_data_val['input_ids']  # Extracting input IDs
attention_masks_val = encoded_data_val['attention_mask']  # Extracting attention masks
labels_val = torch.tensor(data[data.data_type=='val'].label.values)  # Extracting labels and converting to PyTorch tensor

# Creating a TensorDataset for the training set, which combines input IDs, attention masks, and labels
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

# Creating a TensorDataset for the validation set, which combines input IDs, attention masks, and labels
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

# Model Initialization

In [8]:
# Importing the BERT model for sequence classification from the Hugging Face Transformers library
from transformers import BertForSequenceClassification

# Initializing the BERT model for sequence classification
# - "bert-base-uncased": Pre-trained BERT model
# - num_labels: Number of unique labels in the dataset, determined by the length of label_dict
# - output_attentions: Whether to return attentions weights of all layers
# - output_hidden_states: Whether to return hidden states of all layers
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Data Loaders and Optimizer

In [9]:
# Importing necessary classes from torch.utils.data for creating data loaders
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Setting the batch size for training and validation data loaders
batch_size = 3

# Creating a data loader for the training dataset
dataloader_train = DataLoader(dataset_train,  # Training dataset
                              sampler=RandomSampler(dataset_train),  # Random sampler for shuffling
                              batch_size=batch_size)  # Batch size for training

# Creating a data loader for the validation dataset
dataloader_validation = DataLoader(dataset_val,  # Validation dataset
                                   sampler=SequentialSampler(dataset_val),  # Sequential sampler for iterating through the dataset
                                   batch_size=batch_size)  # Batch size for validation

# Importing the AdamW optimizer and the learning rate scheduler from the Hugging Face Transformers library
from transformers import AdamW, get_linear_schedule_with_warmup

# Initializing the AdamW optimizer with the BERT model parameters
# - model.parameters(): Parameters of the BERT model
# - lr: Learning rate (1e-5)
# - eps: Epsilon parameter (small value to avoid division by zero)
optimizer = AdamW(model.parameters(),
                  lr=1e-5,  # Learning rate
                  eps=1e-8)  # Epsilon value for numerical stability

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training Parameters

In [10]:
# Setting the number of epochs for training
epochs = 5

# Initializing the learning rate scheduler
# - get_linear_schedule_with_warmup: Creates a schedule with a learning rate that linearly increases from 0 during warmup steps
# - optimizer: Optimizer to be used (AdamW optimizer in this case)
# - num_warmup_steps: Number of warmup steps (0 in this case, meaning no warmup)
# - num_training_steps: Total number of training steps (number of batches per epoch multiplied by the number of epochs)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

# Model Evaluation Metrics

In [11]:
# Importing necessary function from scikit-learn for F1 score calculation
from sklearn.metrics import f1_score

# Defining a function to calculate F1 score
def f1_score_func(preds, labels):
    # Flatten the predicted and true labels to 1D arrays
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    # Calculate the weighted F1 score
    return f1_score(labels_flat, preds_flat, average='weighted')

# Defining a function to calculate accuracy per class
def accuracy_per_class(preds, labels):
    # Reverse the label dictionary to map label indices to their corresponding labels
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    # Flatten the predicted and true labels to 1D arrays
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    # Iterate over unique labels
    for label in np.unique(labels_flat):
        # Extract predictions and true labels for the current class
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        
        # Print class name, accuracy, and total count of correct predictions for the current class
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Setting Random Seeds and Device

In [12]:
# Importing necessary libraries
import random
import numpy as np

# Setting the seed value for random number generation
seed_val = 17

# Setting the seed for Python's built-in random number generator
random.seed(seed_val)

# Setting the seed for NumPy's random number generator
np.random.seed(seed_val)

# Setting the seed for PyTorch's random number generator
torch.manual_seed(seed_val)

# Setting the seed for PyTorch's CUDA random number generator (if available)
torch.cuda.manual_seed_all(seed_val)

# Checking if GPU is available and assigning the appropriate device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Moving the model to the selected device
model.to(device)

# Printing the selected device (CPU or GPU)
print(device)

cuda


# Evaluation Function

In [13]:
def evaluate(dataloader_val):
    # Set the model to evaluation mode
    model.eval()
    
    # Initialize variables to store total loss, predictions, and true values
    loss_val_total = 0
    predictions, true_vals = [], []
    
    # Iterate over batches in the validation data loader
    for batch in dataloader_val:
        # Move batch to the appropriate device (CPU or GPU)
        batch = tuple(b.to(device) for b in batch)
        
        # Unpack inputs from the batch
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        # Disable gradient calculation during evaluation
        with torch.no_grad():        
            # Forward pass through the model
            outputs = model(**inputs)
            
        # Extract loss and logits from the model outputs
        loss = outputs[0]
        logits = outputs[1]
        
        # Accumulate the total validation loss
        loss_val_total += loss.item()

        # Detach logits from the computation graph and move them to CPU
        logits = logits.detach().cpu().numpy()
        # Move label ids to CPU
        label_ids = inputs['labels'].cpu().numpy()
        
        # Append predictions and true labels to the respective lists
        predictions.append(logits)
        true_vals.append(label_ids)
    
    # Calculate average validation loss
    loss_val_avg = loss_val_total / len(dataloader_val) 
    
    # Concatenate predictions and true labels across all batches
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

# Training Loop

In [14]:
# Iterate through each epoch
for epoch in tqdm(range(1, epochs+1)):
    
    # Set the model to training mode
    model.train()
    
    # Initialize total training loss for the epoch
    loss_train_total = 0

    # Display progress bar for the epoch
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    
    # Iterate through batches in the training data loader
    for batch in progress_bar:
        # Reset gradients
        model.zero_grad()
        
        # Move batch to the appropriate device (CPU or GPU)
        batch = tuple(b.to(device) for b in batch)
        
        # Unpack inputs from the batch
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        # Forward pass through the model
        outputs = model(**inputs)
        
        # Extract loss from the model outputs
        loss = outputs[0]
        loss_train_total += loss.item()
        
        # Backward pass: Compute gradients
        loss.backward()

        # Clip gradients to prevent explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update model parameters
        optimizer.step()
        # Update learning rate scheduler
        scheduler.step()
        
        # Update progress bar with current training loss
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    # Save the model at the end of each epoch
    torch.save(model.state_dict(), f'/kaggle/working/finetuned_BERT_epoch_{epoch}.model')
        
    # Print epoch information
    tqdm.write(f'\nEpoch {epoch}')
    
    # Calculate average training loss for the epoch
    loss_train_avg = loss_train_total / len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    # Evaluate the model on the validation dataset
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    
    # Calculate F1 score on the validation dataset
    val_f1 = f1_score_func(predictions, true_vals)
    
    # Print validation loss and F1 score
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.0034334013159847
Validation loss: 1.2490992045402527
F1 Score (Weighted): 0.9276517419925127


Epoch 2:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.8281914902929689
Validation loss: 0.3915663495659828
F1 Score (Weighted): 0.9538515137069086


...


Epoch 3
Training loss: 0.3226675465178322
Validation loss: 0.22756596557796002
F1 Score (Weighted): 0.9546395465861538


Epoch 4:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.167468023761897
Validation loss: 0.1893121540918946
F1 Score (Weighted): 0.948052395596726


Epoch 5:   0%|          | 0/284 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.11332004249725543
Validation loss: 0.18557060964405536
F1 Score (Weighted): 0.948052395596726


# Model Loading and Evaluation

In [15]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

# Move the model to the specified device (CPU or GPU)
model.to(device)

# Load the fine-tuned weights of the model from the saved file
model.load_state_dict(torch.load('/kaggle/working/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

# Evaluate the model on the validation dataset to obtain predictions and true labels
_, predictions, true_vals = evaluate(dataloader_validation)

# Calculate and print the accuracy for each class based on the predictions and true labels
accuracy_per_class(predictions, true_vals)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Class: Phone
Accuracy: 12/15

Class: Tab
Accuracy: 15/15

Class: TV
Accuracy: 15/15

Class: AC
Accuracy: 15/15

Class: Wash
Accuracy: 13/15

Class: Fridge
Accuracy: 14/15

Class: Vacuum
Accuracy: 13/15

Class: Dish
Accuracy: 14/15

Class: Micro
Accuracy: 13/15

Class: Watch
Accuracy: 15/15



# Inference with Fine-Tuned Model

In [16]:
model.eval()
# Your input text
input_text = "how to fix my washing machine?"

# Tokenize and encode the input text
inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True)
input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

# Make predictions
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=attention_mask)

# Extract predicted probabilities or class labels
predicted_probabilities = torch.softmax(output.logits, dim=1).cpu().numpy()
predicted_class = np.argmax(predicted_probabilities, axis=-1)

# Print the results
print("Predicted probabilities:", predicted_probabilities)
print("Predicted class:", predicted_class)
key_list = list(label_dict.keys())
val_list = list(label_dict.values())
 
# print key with val 100
position = val_list.index(predicted_class)
print(key_list[position])
key_list

Predicted probabilities: [[0.04512616 0.04308594 0.06931993 0.05246408 0.32251826 0.03305347
  0.15799254 0.19748718 0.04669596 0.03225644]]
Predicted class: [4]
Wash


['Phone',
 'Tab',
 'TV',
 'AC',
 'Wash',
 'Fridge',
 'Vacuum',
 'Dish',
 'Micro',
 'Watch']

### After the intent dataset is created from user History, a Hybrid Recommender System leveraging Statistics, Reinforcement Learning and Collaborative Filtering is used to suggest personalized prompt suggestions to the user.

## User Data Generation

### Importing Libraries

In [17]:
# Importing the random module for generating random numbers and performing random sampling
import random

# Importing the NumPy library and aliasing it as np
# NumPy is used for numerical computing and provides support for arrays and mathematical functions
import numpy as np

# Importing the matplotlib.pyplot module and aliasing it as plt
# Matplotlib is a plotting library used to create various types of plots and visualizations
import matplotlib.pyplot as plt

# Importing the pandas library and aliasing it as pd
# Pandas is used for data manipulation and analysis, providing data structures like Series and DataFrame
import pandas as pd

# Generating Synthetic Data

In [18]:
# Assigning the list of possible labels to the variable test_list
test_list = possible_labels

# Printing the original list
print("Original list is : " + str(test_list))

# Generating random labels
y = []
for i in range(10000):
    rand_idx = random.randrange(len(test_list))
    random_label = test_list[rand_idx]
    y.append(random_label)

# Encoding labels using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# Generating random time values
time = []
for i in range(10000):
    time.append(random.randint(0, 23))

# Generating random user IDs
userid = []
for i in range(10000):
    userid.append(random.randint(0, 60))

# Creating a DataFrame with userid, time, query, and reward columns
df = pd.DataFrame({'userid': userid, 'time': time, 'query': y})

# Generating random reward values (0 or 1)
reward = []
for i in range(10000):
    reward.append(random.choice([0,1]))

# Adding the 'reward' column to the DataFrame
df['reward'] = reward

Original list is : ['Phone' 'Tab' 'TV' 'AC' 'Wash' 'Fridge' 'Vacuum' 'Dish' 'Micro' 'Watch']


# Data Overview

In [19]:
# Printing the original list of possible labels
print("Original list is : " + str(test_list))

# Printing the first few rows of the DataFrame
print(df.head())

# Printing the shape of the DataFrame (number of rows and columns)
print(df.shape)

Original list is : ['Phone' 'Tab' 'TV' 'AC' 'Wash' 'Fridge' 'Vacuum' 'Dish' 'Micro' 'Watch']
   userid  time  query  reward
0      38     5      3       0
1      17     5      7       0
2      49    21      8       0
3      46    15      2       0
4      30    19      8       0
(10000, 4)


# User Query Analysis

In [20]:
# Extracting the 'userid' column from the DataFrame
userid=int(input())

# Importing the NumPy library
import numpy as np

# Finding unique labels and their frequencies for the specified 'userid'
unique, frequency = np.unique(df['query'][df['userid']==userid], return_counts = True)

# Sorting indices based on frequencies in descending order
sorted_indices = np.argsort(frequency)[::-1]

# Getting the top two labels and their frequencies
top_labels = unique[sorted_indices[:2]]
top_counts = frequency[sorted_indices[:2]]

# Converting label indices to their original values using inverse transform
top_labeleng=le.inverse_transform(top_labels)
top_labeleng

 3


array(['Wash', 'Dish'], dtype='<U6')

## Contextual Learner Class

### Class Definition

In [21]:
class ContextualLearner:
    # Constructor to initialize the class
    # Parameters:
    #     learnerclass: SGDClassifier (preferred) or SGDRegressor
    #     rew_vec: array of possible rewards for SGDClassifier and None for SGDRegressor
    def __init__(self, learnerclass, rew_vec):
        # Initialize instance variables
        self.sgd = None
        self.hist = 50
        self.arm_sgd = {}
        self.dataX = {}
        self.dataY = {}
        self.rew_vec = rew_vec
        self.Learner = learnerclass

    # Method to learn from an individual datapoint
    # Parameters:
    #     ctx_vector: vector of context values pre-normalized
    #     arm: action selected as a string
    #     reward: scalar reward value
    # Returns:
    #     status: True or False if learning was successful
    def train(self, ctx_vector, arm, reward):
        X = []
        Y = []
        if ctx_vector is None or arm is None or reward is None:
            return False
        # If the arm classifier doesn't exist
        if arm not in self.arm_sgd.keys():
            self.arm_sgd[arm] = self.Learner()
            self.dataX[arm] = []
            self.dataY[arm] = []
        # Get arm classifier and make prediction
        self.sgd = self.arm_sgd[arm]
        if len(self.dataX[arm]) > self.hist:
            X = self.dataX[arm][:-self.hist]
            Y = self.dataY[arm][:-self.hist]
        X.append(ctx_vector)
        X = np.asarray(X)  # .reshape(1, -1)
        Y.append(reward)  # = [reward]
        # Fit the data point
        if self.rew_vec is not None:
            self.sgd.partial_fit(X, Y, self.rew_vec)
        else:
            self.sgd.partial_fit(X, Y)
        # Add to data vectors
        self.dataX[arm].append(ctx_vector)
        self.dataY[arm].append(reward)
        return True

    # Method to predict reward for an individual datapoint
    # Parameters:
    #     ctx_vector: vector of context values pre-normalized
    #     arm: action selected as a string
    # Returns:
    #     reward: scalar reward value
    def predict(self, ctx_vector, arm):
        if ctx_vector is None or arm is None:
            return None
        # If the arm classifier doesn't exist
        if arm in self.arm_sgd.keys():
            # Get arm classifier and make prediction
            self.sgd = self.arm_sgd[arm]
            X = ctx_vector
            X = np.asarray(X).reshape(1, -1)
            return self.sgd.predict(X)[0]
        # If nothing found return
        return 0


In [22]:
print(df)#printing the dataset

      userid  time  query  reward
0         38     5      3       0
1         17     5      7       0
2         49    21      8       0
3         46    15      2       0
4         30    19      8       0
...      ...   ...    ...     ...
9995      55    12      1       0
9996      45    17      0       0
9997       0     0      1       1
9998       5     8      7       1
9999      15     1      0       1

[10000 rows x 4 columns]


# Training Contextual Bandit Learners

### Importing Libraries

In [23]:
# Importing the SGDRegressor and SGDClassifier classes from the sklearn.linear_model module
from sklearn.linear_model import SGDRegressor, SGDClassifier

## Training Bandit Learners

In [24]:

# Assuming df is your DataFrame containing user data

# count unique users
unique_users = df['userid'].unique()

# Define context vector fields
context_vector = ['time']

# Dictionary to hold bandit learners for each user
user_bandits = {}

# Loop over unique users
for user_id in unique_users:
    # Filter DataFrame for current user
    user_df = df[df['userid'] == user_id]
    
    # Initialize a new bandit learner for the current user
    user_bandits[user_id] = ContextualLearner(SGDRegressor, None)
    
    # Loop over records for the current user
    for index, record in user_df.iterrows():
        # Get context vector
        ctx_vec = record[context_vector].tolist()
        # Simple normalization for age
        ctx_vec[0] = ctx_vec[0] / 100.
        # Get recommendation - action or arm
        arm = record['query']
        # Get reward scalar value - rating given by user
        rew = record['reward']
        # Predict reward
        rew_pred = user_bandits[user_id].predict(ctx_vec, arm)
        # Train the bandit learner
        user_bandits[user_id].train(ctx_vec, arm, rew)

## Using Trained Bandit Learners
### Predicting Rewards for Each User at a Specific Time

In [25]:
# Now, you can use the trained bandit learners for each user as needed
# For example, to predict rewards for a specific time for each user:
time = int(input("Enter time: "))
reward_lists = {}  # Dictionary to hold reward lists for each user

for user_id, bandit in user_bandits.items():
    # Assuming df is your DataFrame containing user data
    user_df = df[df['userid'] == user_id]
    
    # Initialize reward list for the current user
    reward_list = []
    
    # Loop over records for the current user
    for index, record in user_df.iterrows():
        # Get context vector
        ctx_vec = record[context_vector].tolist()
        # Simple normalization for age
        ctx_vec[0] = ctx_vec[0] / 100.
        # Get recommendation - action or arm
        arm = record['query']
        # Predict reward
        rew_pred = bandit.predict(ctx_vec, arm)
        reward_list.append(rew_pred)
    
    # Store the reward list for the current user
    reward_lists[user_id] = reward_list

# Now reward_lists dictionary contains reward lists for each user


Enter time:  7


## Getting Top Labels for a User

### Function Definition

In [26]:
def get_top_labels(user_id, num_labels=2):
    # Check if the user ID exists in the bandit learners dictionary
    if user_id in user_bandits:
        # Get the bandit learner for the specified user
        bandit = user_bandits[user_id]
        
        # Predict rewards for all possible actions
        rewards = {}
        num_actions = df['query'].nunique()

        for action in range(num_actions):  # Assuming num_actions is defined somewhere
            # Create a context vector (here, using default values, modify as needed)
            ctx_vec = [0.5]  # Placeholder context vector
            
            # Predict reward for the current action
            rew_pred = bandit.predict(ctx_vec, action)
            rewards[action] = rew_pred
        
        # Sort actions based on predicted rewards
        sorted_actions = sorted(rewards, key=rewards.get, reverse=True)
        
        # Get the top labels and their predicted rewards
        top_labels = [sorted_actions[i] for i in range(min(num_labels, len(sorted_actions)))]
        top_rewards = [rewards[action] for action in top_labels]
        
        return top_labels, top_rewards
    else:
        print("User ID not found.")
        return None, None



## Example Usage

In [27]:
# Example usage: taking user ID input and getting top labels
user_id = int(input("Enter user ID: "))
top_labels, top_rewards = get_top_labels(user_id)

if top_labels is not None:
    print("Top labels for user {}: {}".format(user_id, top_labels))
    print("Corresponding rewards:", top_rewards)

Enter user ID:  4


Top labels for user 4: [0, 8]
Corresponding rewards: [0.08106802524370878, 0.05794403900292082]


# Collaborative Filtering Method

### Splitting Data for Training and Testing

In [28]:
# Importing the train_test_split function from the sklearn.model_selection module
from sklearn.model_selection import train_test_split

# Splitting the DataFrame 'df' into train and test sets
# Parameters:
#     df: DataFrame to be split
#     test_size: proportion of the dataset to include in the test split (0.30 indicates 30%)
#     random_state: seed used by the random number generator
X_train, X_test = train_test_split(df, test_size=0.30, random_state=42)

# Printing the shapes of the train and test sets
print(X_train.shape)
print(X_test.shape)

(7000, 4)
(3000, 4)


### Creating User-Item Matrix

In [29]:
# Creating a pivot table from the training data
# Parameters:
#     index: Column to use as the index in the pivot table ('userid' in this case)
#     columns: Column to use as the columns in the pivot table ('query' in this case)
#     values: Column to use as the values in the pivot table ('time' in this case)
#     aggfunc: Aggregation function to apply if there are multiple values for a given index/column pair ('mean' in this case)
#     fillna: Value to replace NaN values with (0 in this case)
user_data = X_train.pivot_table(index='userid', columns='query', values='time', aggfunc='mean').fillna(0)

# Displaying the first few rows of the pivot table
user_data.head()

query,0,1,2,3,4,5,6,7,8,9
userid,,,,,,,,,,
0,15.375000,8.454545,10.545455,12.000000,10.083333,16.777778,7.900000,15.250000,8.222222,14.461538
1,8.214286,9.916667,10.181818,13.666667,9.200000,12.888889,13.214286,10.700000,9.818182,9.750000
2,11.466667,10.444444,16.066667,8.333333,14.625000,13.647059,10.214286,13.875000,12.357143,9.666667
3,8.833333,10.428571,8.416667,11.875000,12.700000,10.846154,14.277778,10.285714,12.444444,10.000000
4,11.333333,8.200000,7.333333,9.000000,8.500000,8.250000,10.583333,12.285714,10.733333,10.750000


### Preparing Dummy Train and Test Data

In [30]:
# Creating copies of the training and testing datasets
dummy_train = X_train.copy()
dummy_test = X_test.copy()

# Transforming time values to binary (0 if > 0 else 1)
dummy_train['time'] = dummy_train['time'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['time'] = dummy_test['time'].apply(lambda x: 1 if x > 0 else 0)

# Marking unrated items as 1 for prediction and 0 for evaluation
dummy_train = dummy_train.pivot_table(index='userid', columns='query', values='time', aggfunc='mean').fillna(1)
dummy_test = dummy_test.pivot_table(index='userid', columns='query', values='time', aggfunc='mean').fillna(0)

# Displaying the first few rows of the dummy train and test datasets
dummy_train.head()
dummy_test.head()

query,0,1,2,3,4,5,6,7,8,9
userid,,,,,,,,,,
0,1.000000,1.000000,1.000000,1.0,1.000000,0.923077,0.8,1.0,1.0,1.0
1,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0
2,1.000000,1.000000,1.000000,1.0,0.857143,1.000000,1.0,1.0,0.0,1.0
3,1.000000,0.888889,0.500000,1.0,1.000000,1.000000,1.0,1.0,1.0,1.0
4,0.916667,1.000000,0.833333,1.0,1.000000,0.750000,1.0,1.0,1.0,1.0


### Computing User Similarity Matrix

In [31]:
# Importing the cosine_similarity function from the sklearn.metrics.pairwise module
from sklearn.metrics.pairwise import cosine_similarity

# Computing cosine similarity between users using the user data
user_similarity = cosine_similarity(user_data)

# Handling NaN values by replacing them with 0
user_similarity[np.isnan(user_similarity)] = 0

# Printing the user similarity matrix and its shape
print("User Similarity Matrix:")
print(user_similarity)
print("Shape:", user_similarity.shape)

User Similarity Matrix:
[[1.         0.95320026 0.95348032 ... 0.9633914  0.90903211 0.97131143]
 [0.95320026 1.         0.95894915 ... 0.97724512 0.93504682 0.98445468]
 [0.95348032 0.95894915 1.         ... 0.97180626 0.93146521 0.97841628]
 ...
 [0.9633914  0.97724512 0.97180626 ... 1.         0.9639072  0.97513513]
 [0.90903211 0.93504682 0.93146521 ... 0.9639072  1.         0.93540072]
 [0.97131143 0.98445468 0.97841628 ... 0.97513513 0.93540072 1.        ]]
Shape: (61, 61)


##  Predicting Ratings

In [32]:
# Predict user ratings by dot product of similarity matrix and user-item matrix
user_predicted_ratings = np.dot(user_similarity, user_data)
user_predicted_ratings


array([[700.40420435, 643.44148028, 683.61330982, 650.20329654,
        660.39287842, 657.57399215, 649.24663133, 653.86791447,
        667.22382139, 659.77736817],
       [710.96065403, 654.98870672, 696.18997277, 662.04174527,
        671.965631  , 668.16707267, 661.87696198, 664.43177826,
        679.03027287, 669.9091367 ],
       [710.75945639, 654.3208316 , 696.23687517, 660.08045119,
        672.34443029, 667.1097069 , 660.53661069, 664.1765844 ,
        678.48786762, 668.57618222],
       [712.2420941 , 656.20450288, 696.43724802, 662.32151454,
        673.69127775, 668.20614991, 662.82220758, 665.10939312,
        680.28738123, 670.62193998],
       [714.90160073, 657.23936813, 697.45870269, 663.27855653,
        674.15241621, 669.09528009, 663.63282287, 667.19440475,
        681.70328677, 672.64563372],
       [717.56369458, 660.85587915, 700.9939619 , 666.15397202,
        677.7217793 , 671.61911486, 665.86737997, 669.02278495,
        684.32589711, 675.68399867],
       [72

### Finalizing Predictions

In [33]:
# Multiply predicted ratings with the dummy train data
user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()


query,0,1,2,3,4,5,6,7,8,9
userid,,,,,,,,,,
0,0.000000,58.494680,0.000000,72.244811,55.032740,0.0,0.0,54.488993,74.135980,50.752105
1,152.348712,0.000000,63.289998,0.000000,44.797709,0.0,0.0,0.000000,61.730025,0.000000
2,47.383964,72.702315,0.000000,0.000000,0.000000,0.0,0.0,41.511037,0.000000,44.571745
3,118.707016,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,37.793743,83.827742
4,79.433511,0.000000,464.972468,0.000000,0.000000,0.0,0.0,0.000000,90.893772,112.107606


### Example: Top recommendations for a user


In [34]:
# Specify the user ID for which recommendations are to be generated
user_id = 42

# Get the top recommendations for the specified user
top_recommendations = user_final_ratings.iloc[user_id].sort_values(ascending=False)[:5]

# Print the top recommendations for the user
print("Top recommendations for user", user_id, ":", top_recommendations)

Top recommendations for user 42 : query
3    59.250602
7    43.651413
1     0.000000
0     0.000000
4     0.000000
Name: 42, dtype: float64


#### *Although, the label classifier requires GPU to train, the core recommender is very lightweight, and  can be run on CPU.*

In [39]:
# Get the index of the top recommendation
top_label_index = top_recommendations.index[0]

# Print the index of the top recommendation
print(top_label_index)

3

In [48]:
# Filter the DataFrame 'data' to include only rows where the 'label' column matches the top label index
filtered_df = data[data['label'] == top_label_index]

In [49]:
# Randomly select a recommendation from the filtered DataFrame
recommendation = filtered_df.sample(n=1)['Input'].iloc[0]

# Print the selected recommendation
print(recommendation)

Set a schedule for the AC to turn off at 9 PM.


### **Future Plans**: We plan to add full device management capabilities to our chatbot, and not just stop at personalized recommendations. We tinkered with integration of NodeRED with out chatbot, and saw some promising possibilties. As our backbone LLMs are very powerful, if given more time, we would love to add full device management capabilities to our system. 